In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model,save_model
import tensorflow as tf
import numpy as np
from numpy import array
import pandas as pd
from sklearn.model_selection import train_test_split
import string
import re
import nltk
from nltk.stem import WordNetLemmatizer
import string
from nltk import word_tokenize
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
#stop = stopwords.words('english')
from tensorflow.keras.utils import to_categorical

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
stop_words = stopwords.words('english')
addition = ['•', '!', '"', '#', '”', '“', '$', '%', '&', "'", '–', '(', ')', '*','’', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '…']
stop_words.extend(addition)

In [3]:
data = pd.read_csv('Reviews.csv')

In [4]:
data.dropna(how='any',inplace=True)
data.drop_duplicates(inplace=True, subset=['Score','Text'])
idx = data[data["HelpfulnessNumerator"]>data["HelpfulnessDenominator"]].index
data.drop(index=idx, inplace=True)

In [5]:
def create_target(x):
    return 2 if x>3 else 0 if x<3 else 1
data['target'] = data['Score'].apply(create_target)

In [6]:
neutral = data.loc[data.target==1]
positive = data.loc[data.target==2].sample(40000)
negative = data.loc[data.target==0].sample(40000)
data = pd.concat([positive, negative, neutral])

In [7]:
def preprocess(text):
  text = text.lower()
  text = re.sub(r'\@\w+|\#','', text)
  text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
  text = text.translate(str.maketrans('', '', string.punctuation))
  result = ''.join([i for i in text if not i.isdigit()])
  tweet_tokens = nltk.word_tokenize(result)
  filtered_words = [w for w in tweet_tokens if not w in stop_words]
  lemmatizer = WordNetLemmatizer()
  lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in filtered_words]
  lemma_words=" ".join(lemma_words)
  return lemma_words

In [8]:
data['cleaned'] = data['Text'].apply(preprocess)

In [9]:
data.cleaned.apply(lambda x: len(x.split(" "))).mean()

43.166256718593424

In [10]:
text = data['cleaned'].tolist()
y = data['target']

In [11]:
token = Tokenizer()
token.fit_on_texts(text)

In [12]:
vocab_size  = len(token.word_index) + 1

In [13]:
encoded_text = token.texts_to_sequences(text)

In [14]:
max_length = 50
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')

In [15]:
glove_vectors = dict()

In [16]:
file = open('glove.6B.50d.txt', encoding='utf-8')

for line in file:
    values = line.split()
    word = values[0]
    #storing the word in the variable
    vectors = np.asarray(values[1: ])
    #storing the vector representation of the respective word in the dictionary
    glove_vectors[word] = vectors
file.close()

In [17]:
keys = glove_vectors.keys()

In [18]:
word_vector_matrix = np.zeros((vocab_size, 50))
for word, index in token.word_index.items():
    vector = glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector

In [19]:
y_cat = to_categorical(y)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, random_state = 42, test_size = 0.3, stratify = y_cat)

In [21]:
model =  tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 50, input_length=max_length, weights = [word_vector_matrix], trainable = False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3,activation = 'softmax')
])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            5119850   
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 256)           183296    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               164352    
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 195       
Total params: 5,475,949
Trainable params: 356,099
Non-trainable params: 5,119,850
________________________________________

In [23]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
             optimizer=tf.keras.optimizers.Adam(1e-4), 
             metrics=['accuracy'])

In [24]:
history = model.fit(x = X_train, y = y_train, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
2402/2402 [==============================] - 253s 105ms/step - loss: 0.9821 - accuracy: 0.5459 - val_loss: 0.9496 - val_accuracy: 0.5787
Epoch 2/100
2402/2402 [==============================] - 268s 111ms/step - loss: 0.9432 - accuracy: 0.5871 - val_loss: 0.9360 - val_accuracy: 0.5925
Epoch 3/100
2402/2402 [==============================] - 245s 102ms/step - loss: 0.9286 - accuracy: 0.6021 - val_loss: 0.9212 - val_accuracy: 0.6099
Epoch 4/100
2402/2402 [==============================] - 245s 102ms/step - loss: 0.9167 - accuracy: 0.6159 - val_loss: 0.9130 - val_accuracy: 0.6180
Epoch 5/100
2402/2402 [==============================] - 249s 104ms/step - loss: 0.9092 - accuracy: 0.6241 - val_loss: 0.9098 - val_accuracy: 0.6209
Epoch 6/100
2402/2402 [==============================] - 246s 102ms/step - loss: 0.9014 - accuracy: 0.6334 - val_loss: 0.9029 - val_accuracy: 0.6284
Epoch 7/100
2402/2402 [==============================] - 242s 101ms/step - loss: 0.8958 - accuracy: 0.6386

2402/2402 [==============================] - 233s 97ms/step - loss: 0.7444 - accuracy: 0.8057 - val_loss: 0.8974 - val_accuracy: 0.6473
Epoch 56/100
2402/2402 [==============================] - 231s 96ms/step - loss: 0.7423 - accuracy: 0.8077 - val_loss: 0.8992 - val_accuracy: 0.6456
Epoch 57/100
2402/2402 [==============================] - 234s 98ms/step - loss: 0.7402 - accuracy: 0.8099 - val_loss: 0.8998 - val_accuracy: 0.6452
Epoch 58/100
2402/2402 [==============================] - 231s 96ms/step - loss: 0.7386 - accuracy: 0.8115 - val_loss: 0.9023 - val_accuracy: 0.6427
Epoch 59/100
2402/2402 [==============================] - 233s 97ms/step - loss: 0.7393 - accuracy: 0.8106 - val_loss: 0.8980 - val_accuracy: 0.6480
Epoch 60/100
2402/2402 [==============================] - 232s 97ms/step - loss: 0.7382 - accuracy: 0.8119 - val_loss: 0.8998 - val_accuracy: 0.6453
Epoch 61/100
2402/2402 [==============================] - 234s 98ms/step - loss: 0.7359 - accuracy: 0.8142 - val_loss: 

In [25]:
y = model.evaluate(x = X_test, y = y_test)
y[1]

1030/1030 [==============================] - 26s 26ms/step - loss: 0.9009 - accuracy: 0.6466


0.6465640068054199